In [1]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../../'))

from config.config_managers import DashboardConfigManager
from dataManager import DataManager
from dash import Dash
import pandas as pd
import plotly.express as px
from abc import ABC, abstractmethod
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

2024-12-15 16:37:17 - INFO - PyTorch version 2.2.2 available.


In [2]:
CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/dashboard-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)
dash_data = data_manager.load_data()

2024-12-15 16:37:19 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/analysis/My Drive
2024-12-15 16:37:19 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-15 16:37:19 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/18 [00:00<?, ?it/s]

2024-12-15 16:37:25 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/analysis/My Drive
2024-12-15 16:37:25 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-12-15 16:37:25 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert


  0%|          | 0/18 [00:00<?, ?it/s]

In [56]:


class Visualization(ABC):
    def __init__(self, data, mappings):
        self.data = data
        self.tag_mapping = mappings['tag_mapping']
        self.dataset_mapping = mappings['dataset_mapping']

    @abstractmethod
    def prepare_data(self):
        pass

    @abstractmethod
    def visualize(self):
        pass

    def replace_mappings(self, data):
        data['Tag'] = data['Tag'].replace(self.tag_mapping)
        data['Dataset'] = data['Dataset'].replace(self.dataset_mapping)
        return data


mappings = {
    'tag_mapping': {'PERS': 'PER'},
    'dataset_mapping': {'ANERCorp_CamelLab_arabertv02': 'ANERCorp', 'conll2003_bert': 'CoNLL-2003'}
}

class ReportBarChart(Visualization):
    def prepare_data(self):
        report_data = []
        for data_name, data_content in self.data.items():
            entity_report = data_content.entity_non_strict_report
            entity_strict_report = data_content.entity_strict_report
            entity_report['Dataset'] = data_name
            entity_report['Scheme'] = 'IOB'
            entity_strict_report['Dataset'] = data_name
            entity_strict_report['Scheme'] = 'IOB2'
            report_data.append(pd.concat([
                entity_report, 
                entity_strict_report
            ]))
        report_df = pd.concat(report_data)
        report_df = report_df[~report_df['Tag'].isin(['micro', 'macro', 'weighted'])]
        
        return self.replace_mappings(report_df)

    def visualize(self):
        entity_report_data = self.prepare_data()
        melted_df = entity_report_data.melt(id_vars=["Tag", "Support", "Dataset", "Scheme"], 
                        value_vars=["Precision", "Recall"], 
                        var_name="Metric", value_name="Value")
        melted_df['Value'] = melted_df['Value'].round(3)
        fig = px.bar(melted_df, x="Tag", y="Value",
                    facet_row="Scheme", facet_col="Dataset",
                    title="Breakdown of Precision and Recall Scores by Entity Tag, Categorized by Dataset and Tagging Scheme",
                    labels={"Value": "Score", 'Tag': 'Entity'},
                    color="Metric", barmode="group",
                    template="plotly_white",
                    facet_row_spacing=0.15,  # Adjust to a higher value for more space
                    facet_col_spacing=0.1,  # Adjust to a higher value for more space
                    text='Value',  # Display the Value on top of each bar
                    )

        fig.show()

class ConfusionBarChart(Visualization):
    def prepare_data(self):
        matrix_data = []
        for data_name, data_content in self.data.items():
            entity_matrix = pd.DataFrame(data_content.entity_non_strict_confusion_data['confusion_matrix']).T 
            entity_strict_matrix = pd.DataFrame(data_content.entity_strict_confusion_data['confusion_matrix']).T
            entity_matrix['Dataset'] = data_name
            entity_matrix['Scheme'] = 'IOB'
            entity_strict_matrix['Dataset'] = data_name
            entity_strict_matrix['Scheme'] = 'IOB2'
            matrix_data.append(pd.concat([
				entity_matrix, 
				entity_strict_matrix
			]))
        matrix_df = pd.concat(matrix_data)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        matrix_data = self.replace_mappings(matrix_df)
        
        grouped = matrix_data.groupby(['Tag', 'Dataset', 'Scheme']).sum()
        grouped['Total'] = grouped['TP'] + grouped['FP'] + grouped['FN']
        
        matrix_data = matrix_data.merge(grouped['Total'], on=['Tag', 'Dataset', 'Scheme'], how='left')
        
        matrix_data['TP_Count'] = matrix_data['TP']
        matrix_data['FP_Count'] = matrix_data['FP']
        matrix_data['FN_Count'] = matrix_data['FN']
        
        matrix_data['TP'] = matrix_data['TP'] / matrix_data['Total']
        matrix_data['FP'] = matrix_data['FP'] / matrix_data['Total']
        matrix_data['FN'] = matrix_data['FN'] / matrix_data['Total']
        
        confusion_scaled_df = matrix_data.melt(id_vars=["Tag", "Dataset", "Scheme"], value_vars=["TP", "FP", "FN"], var_name="Metric", value_name="Scale")
        confusion_count_df = matrix_data.melt(id_vars=["Tag", "Dataset", "Scheme"], value_vars=["TP_Count", "FP_Count", "FN_Count"], var_name="Metric", value_name="Count")
        confusion_count_df['Metric'] = confusion_count_df['Metric'].str.replace('_Count', '')
        confusion_data = confusion_scaled_df.merge(confusion_count_df, on=["Tag", "Dataset", "Scheme", "Metric"])
        return confusion_data

    def visualize(self):
        confusion_df = self.prepare_data()
        
        fig = px.bar(confusion_df, x="Tag", y="Scale", color="Metric",
            facet_row="Scheme", facet_col="Dataset",
            title="Breakdown of Confusion Matrix Components: True Positives, False Positives, and False Negatives by Entity Tag, Categorized by Dataset and Tagging Scheme",
            labels={"Scale": "Scaled Counts"},
            barmode='group',
            template="plotly_white",
            facet_row_spacing=0.1,  # Adjusted spacing
            facet_col_spacing=0.08,
            text='Count'  # Display the actual Count on top of each bar
            )
        
        fig.show()

class ConfusionHeatmap(Visualization):
    def prepare_data(self):
        matrix_data = []
        for data_name, data_content in self.data.items():
            entity_matrix = pd.DataFrame(data_content.entity_non_strict_confusion_data['confusion_matrix']).T 
            entity_strict_matrix = pd.DataFrame(data_content.entity_strict_confusion_data['confusion_matrix']).T
            entity_matrix['Dataset'] = data_name
            entity_matrix['Scheme'] = 'IOB'
            entity_strict_matrix['Dataset'] = data_name
            entity_strict_matrix['Scheme'] = 'IOB2'
            matrix_data.append(pd.concat([
				entity_matrix, 
				entity_strict_matrix
			]))
        matrix_df = pd.concat(matrix_data)
        matrix_df.reset_index(inplace=True)
        matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
        return self.replace_mappings(matrix_df)

    def visualize(self):
        matrix_df = self.prepare_data()
        confusion_df = matrix_df.melt(id_vars=['Tag', 'Dataset', 'Scheme'], value_vars=['TP', 'FP', 'FN'], 
                            var_name='Metric', value_name='Count')
        
        unique_schemes = confusion_df['Scheme'].unique()
        unique_datasets = confusion_df['Dataset'].unique()
        
        fig = make_subplots(rows=len(unique_schemes), cols=len(unique_datasets),
                            subplot_titles=[f"{dataset} - {scheme}" for scheme in unique_schemes for dataset in unique_datasets],
                            shared_yaxes=True, horizontal_spacing=0.02, vertical_spacing=0.1)
        
        max_value = confusion_df['Count'].max()
        
        for idx, scheme in enumerate(unique_schemes):
            for jdx, dataset in enumerate(unique_datasets):
                filtered_data = confusion_df[(confusion_df['Scheme'] == scheme) & (confusion_df['Dataset'] == dataset)]
                heatmap_data = filtered_data.pivot_table(index='Metric', columns='Tag', values='Count', fill_value=0)
                text_data = filtered_data.pivot_table(index='Metric', columns='Tag', values='Count', fill_value=0).astype(int)

                
                
                fig.add_trace(
                    go.Heatmap(
                        z=heatmap_data,
                        x=heatmap_data.columns,
                        y=heatmap_data.index,
                        colorscale='RdBu_r',
                        coloraxis="coloraxis",  # Use a unified color axis
                        text=text_data,  # Add text annotations
                        texttemplate="%{text}",  # Use the text values directly
                        hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
                    ),
                    row=idx + 1, col=jdx + 1
                )
                
        fig.update_layout(
            coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
            title_text="Confusion Matrix Heatmap Categorized by Dataset and Tagging Scheme",
            template="plotly_white",
            height=700, width=700,
        )
        fig.show()



In [57]:
# Confusion matrix heatmap example
report_bar = ReportBarChart(dash_data, mappings)
report_bar.visualize()

In [54]:
confusion_bar = ConfusionBarChart(dash_data, mappings)
confusion_bar.visualize()

In [ ]:
# Confusion matrix heatmap example
confusion_heatmap = ConfusionHeatmap(dash_data, mappings)
confusion_heatmap.visualize()

# Debug Confusion

In [9]:
dataset_name = 'ANERCorp_CamelLab'
model_name = 'arabertv02'
base_path = Path(f"/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/{dataset_name}_{model_name}/extractions")
df = pd.read_json(
	base_path / 'results/non_strict_entity_misclassifications.json',
	orient='index'
)

In [18]:
entity = 'LOC'
print(df[entity].sum()) 
print(df.loc[entity].sum())


74
42


In [22]:
df

,O,PERS,MISC,LOC,ORG
O,0,40,24,16,64
PERS,64,0,7,9,20
MISC,116,6,0,19,26
LOC,32,2,3,0,5
ORG,99,23,9,30,0


In [93]:
false_positives': {'LOC': {'O': 15,
   'Boundary': 14,
   'Entity and Boundary': 15,
   'MISC': 6,
   'PERS': 6,
   'ORG': 20},

58

In [91]:
32+2+3+5

42

In [25]:
ad = dash_data['ANERCorp_CamelLab_arabertv02'].analysis_data

In [95]:
fad = ad[ad['Labels'] !=-100].copy()
loc_fad = fad[fad['Pred Entities'] == 'LOC'].copy()
loc_fad[loc_fad['True Entities'] == 'ORG']

,Sentence Ids,Token Positions,Words,Tokens,Word Pieces,Core Tokens,True Labels,Token Selector Id,Pred Labels,Agreements,...,Strict Pred Entities,True Entities,Pred Entities,True Aligned Scheme,Pred Aligned Scheme,Consistency Ratio,Inconsistency Ratio,Normalized Token Entropy,Normalized Word Entropy,Normalized Prediction Entropy
762,20,44,المتحدة,المتحدة,المتحدة,المتحدة,B-ORG,المتحدة@#44@#20,I-LOC,False,...,LOC,ORG,LOC,True,True,0.014085,0.985915,0.512727,0.512727,0.048109
1384,40,34,الأوسط,الأوسط,الأوسط,الأوسط,I-ORG,الأوسط@#34@#40,I-LOC,False,...,O,ORG,LOC,True,False,0.115385,0.884615,0.391350,0.391350,0.442727
8241,261,34,الاناضول,الاناضول,الاناضول,الاناضول,B-ORG,الاناضول@#34@#261,B-LOC,False,...,LOC,ORG,LOC,True,True,1.000000,0.000000,0.000000,0.000000,0.032935
8481,268,34,الاناضول,الاناضول,الاناضول,الاناضول,B-ORG,الاناضول@#34@#268,B-LOC,False,...,LOC,ORG,LOC,True,True,1.000000,0.000000,0.000000,0.000000,0.032272
10207,320,18,المتحدة,المتحدة,المتحدة,المتحدة,B-ORG,المتحدة@#18@#320,I-LOC,False,...,LOC,ORG,LOC,True,True,0.014085,0.985915,0.512727,0.512727,0.027225
10305,323,10,اندرو,اندرو,اندرو,اندرو,I-ORG,اندرو@#10@#323,I-LOC,False,...,O,ORG,LOC,True,False,0.000000,1.000000,0.918300,0.918300,0.308748
17100,561,28,إسرائيل,إسرائيل,إسرائيل,إسرائيل,I-ORG,إسرائيل@#28@#561,B-LOC,False,...,LOC,ORG,LOC,True,True,0.000000,1.000000,0.088100,0.088100,0.098931
17635,575,3,الأوروبي,الأوروبي,الأوروبي,الأوروبي,I-ORG,الأوروبي@#3@#575,I-LOC,False,...,O,ORG,LOC,True,False,0.318182,0.681818,0.902400,0.902400,0.390927
18949,618,39,الأزهر,الأزهر,الأزهر,الأزهر,B-ORG,الأزهر@#39@#618,B-LOC,False,...,LOC,ORG,LOC,True,True,0.000000,0.000000,-1.000000,-1.000000,0.577842
21068,694,1,مدريد,مدريد,مدريد,مدريد,I-ORG,مدريد@#1@#694,B-LOC,False,...,LOC,ORG,LOC,False,True,0.500000,0.500000,1.000000,1.000000,0.161362


In [77]:
fad[(fad['True Labels'] == 'I-LOC') & (fad['Pred Labels'] == 'B-LOC') | (fad['True Labels'] == 'B-LOC') & (fad['Pred Labels'] == 'I-LOC')]

,Sentence Ids,Token Positions,Words,Tokens,Word Pieces,Core Tokens,True Labels,Token Selector Id,Pred Labels,Agreements,...,Strict Pred Entities,True Entities,Pred Entities,True Aligned Scheme,Pred Aligned Scheme,Consistency Ratio,Inconsistency Ratio,Normalized Token Entropy,Normalized Word Entropy,Normalized Prediction Entropy
1819,44,11,العراق,العراق,العراق,العراق,I-LOC,العراق@#11@#44,B-LOC,False,...,LOC,LOC,LOC,True,True,0.000000,1.000000,0.000000,0.000000,0.062715
2844,75,16,الصين,الصين,الصين,الصين,I-LOC,الصين@#16@#75,B-LOC,False,...,LOC,LOC,LOC,True,True,0.009804,0.990196,0.100189,0.100189,0.311366
6292,192,6,افريقيا,افريقيا,افريقيا,افريقيا,I-LOC,افريقيا@#6@#192,B-LOC,False,...,LOC,LOC,LOC,True,True,0.000000,1.000000,0.000000,0.000000,0.194391
9975,312,24,والقصور,والقصور,والقصور,والقصور,I-LOC,والقصور@#24@#312,B-LOC,False,...,LOC,LOC,LOC,True,True,0.000000,0.000000,-1.000000,-1.000000,0.375469
15382,504,7,تشيكوسلوفاكيا,تشيك,"تشيك, ##وسل, ##وفاكيا",تشيك,I-LOC,تشيك@#7@#504,B-LOC,False,...,LOC,LOC,LOC,False,True,0.000000,0.000000,-1.000000,-1.000000,0.159374
18794,614,25,وأوسلو,وأ,"وأ, ##وسل, ##و",وأ,I-LOC,وأ@#25@#614,B-LOC,False,...,LOC,LOC,LOC,True,True,0.000000,1.000000,0.918300,0.918300,0.066027
19087,623,28,بألمانيا,بألمانيا,بألمانيا,بألمانيا,I-LOC,بألمانيا@#28@#623,B-LOC,False,...,LOC,LOC,LOC,False,True,0.000000,1.000000,0.000000,0.000000,0.013218
20665,678,9,جاندولفو,جان,"جان, ##دو, ##لف, ##و",جان,B-LOC,جان@#9@#678,I-LOC,False,...,O,LOC,LOC,True,False,0.000000,1.000000,0.650000,0.650000,0.719455


In [89]:
fad[(fad['True Labels'] == 'I-LOC') & (fad['Pred Labels'] == 'O') ]

,Sentence Ids,Token Positions,Words,Tokens,Word Pieces,Core Tokens,True Labels,Token Selector Id,Pred Labels,Agreements,...,Strict Pred Entities,True Entities,Pred Entities,True Aligned Scheme,Pred Aligned Scheme,Consistency Ratio,Inconsistency Ratio,Normalized Token Entropy,Normalized Word Entropy,Normalized Prediction Entropy
2928,79,1,الهندي,الهندي,الهندي,الهندي,I-LOC,الهندي@#1@#79,O,False,...,O,LOC,O,False,True,0.727273,0.272727,0.845400,0.845400,0.000757
10064,315,29,الأمير,الأمير,الأمير,الأمير,I-LOC,الأمير@#29@#315,O,False,...,O,LOC,O,True,True,0.000000,1.000000,0.297500,0.297500,0.029843
10190,320,1,المتحدة,المتحدة,المتحدة,المتحدة,I-LOC,المتحدة@#1@#320,O,False,...,O,LOC,O,False,True,0.334507,0.665493,0.512727,0.512727,0.007918
11968,382,1,المتحدة,المتحدة,المتحدة,المتحدة,I-LOC,المتحدة@#1@#382,O,False,...,O,LOC,O,False,True,0.334507,0.665493,0.512727,0.512727,0.360327
13750,445,1,الأطلسي,الأطلسي,الأطلسي,الأطلسي,I-LOC,الأطلسي@#1@#445,O,False,...,O,LOC,O,False,True,0.500000,0.500000,1.000000,1.000000,0.098079
17616,574,1,الأسود,الأسود,الأسود,الأسود,I-LOC,الأسود@#1@#574,O,False,...,O,LOC,O,False,True,0.041667,0.958333,0.313691,0.313691,0.000883
22283,731,1,المتحدة,المتحدة,المتحدة,المتحدة,I-LOC,المتحدة@#1@#731,O,False,...,O,LOC,O,False,True,0.334507,0.665493,0.512727,0.512727,0.007824
29142,948,25,المكرمة,المكرمة,المكرمة,المكرمة,I-LOC,المكرمة@#25@#948,O,False,...,O,LOC,O,True,True,0.000000,1.000000,0.000000,0.000000,0.009811


In [ ]:
fad[(fad['True Labels'] == 'B-LOC') & (fad['Pred Labels'] == 'O') ]

,Sentence Ids,Token Positions,Words,Tokens,Word Pieces,Core Tokens,True Labels,Token Selector Id,Pred Labels,Agreements,...,Strict Pred Entities,True Entities,Pred Entities,True Aligned Scheme,Pred Aligned Scheme,Consistency Ratio,Inconsistency Ratio,Normalized Token Entropy,Normalized Word Entropy,Normalized Prediction Entropy
1,0,1,الصالحية,الصالحية,الصالحية,الصالحية,B-LOC,الصالحية@#1@#0,O,False,...,O,LOC,O,True,True,0.000000,0.000000,-1.000000,-1.000000,0.409098
4047,116,1,مارون,مارون,مارون,مارون,B-LOC,مارون@#1@#116,O,False,...,O,LOC,O,True,True,1.000000,0.000000,0.000000,0.000000,0.001293
4056,116,10,قطرية,قطرية,قطرية,قطرية,B-LOC,قطرية@#10@#116,O,False,...,O,LOC,O,True,True,0.000000,0.000000,-1.000000,-1.000000,0.000852
4295,124,1,العراق,العراق,العراق,العراق,B-LOC,العراق@#1@#124,O,False,...,O,LOC,O,True,True,1.000000,0.000000,0.000000,0.000000,0.243604
4561,130,1,فيينا,فيينا,فيينا,فيينا,B-LOC,فيينا@#1@#130,O,False,...,O,LOC,O,True,True,1.000000,0.000000,0.000000,0.000000,0.001546
5839,169,40,الجزيرة,الجزيرة,الجزيرة,الجزيرة,B-LOC,الجزيرة@#40@#169,O,False,...,O,LOC,O,True,True,0.203390,0.796610,0.897950,0.897950,0.020915
6167,183,3,قطر,قطر,قطر,قطر,B-LOC,قطر@#3@#183,O,False,...,O,LOC,O,True,True,0.740741,0.259259,0.662082,0.662082,0.341777
6291,192,5,جنوب,جنوب,جنوب,جنوب,B-LOC,جنوب@#5@#192,O,False,...,O,LOC,O,True,True,0.054348,0.945652,0.304600,0.304600,0.337172
7330,232,1,وبأفريقيا,وبأ,"وبأ, ##فريقيا",وبأ,B-LOC,وبأ@#1@#232,O,False,...,O,LOC,O,True,True,0.500000,0.500000,1.000000,1.000000,0.136408
7538,238,1,البحر,البحر,البحر,البحر,B-LOC,البحر@#1@#238,O,False,...,O,LOC,O,True,True,0.458333,0.541667,0.761451,0.761451,0.067573


In [83]:
fad[fad['Sentence Ids']  == 315]

,Sentence Ids,Token Positions,Words,Tokens,Word Pieces,Core Tokens,True Labels,Token Selector Id,Pred Labels,Agreements,...,Strict Pred Entities,True Entities,Pred Entities,True Aligned Scheme,Pred Aligned Scheme,Consistency Ratio,Inconsistency Ratio,Normalized Token Entropy,Normalized Word Entropy,Normalized Prediction Entropy
10036,315,1,للملكية,للملكية,للملكية,للملكية,I-ORG,للملكية@#1@#315,O,False,...,O,ORG,O,False,True,1.000000,0.000000,0.000000,0.000000,0.001451
10037,315,2,هذه,هذه,هذه,هذه,O,هذه@#2@#315,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.000568
10038,315,3,المادة,المادة,المادة,المادة,O,المادة@#3@#315,O,True,...,O,O,O,True,True,0.000000,0.000000,-1.000000,-1.000000,0.001924
10039,315,4,المتفجرة,المتفجرة,المتفجرة,المتفجرة,O,المتفجرة@#4@#315,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.001009
10040,315,5,التي,التي,التي,التي,O,التي@#5@#315,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.000252
10041,315,6,كانت,كانت,كانت,كانت,O,كانت@#6@#315,O,True,...,O,O,O,True,True,0.994118,0.005882,0.052000,0.052000,0.000252
10042,315,7,مدفونة,مدف,"مدف, ##ونة",مدف,O,مدف@#7@#315,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.000473
10044,315,9,تحت,تحت,تحت,تحت,O,تحت@#9@#315,O,True,...,O,O,O,True,True,0.976744,0.023256,0.159400,0.159400,0.000599
10045,315,10,الأرض,الأرض,الأرض,الأرض,O,الأرض@#10@#315,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.001041
10046,315,11,عددا,عددا,عددا,عددا,O,عددا@#11@#315,O,True,...,O,O,O,True,True,1.000000,0.000000,0.000000,0.000000,0.000789


In [24]:
false_negatives': {'LOC': {'O': 29,
   'Boundary': 10,
   'PERS': 1,
   'MISC': 3,
   'ORG': 3,
   'Entity and Boundary': 3},

752

# Development

report

In [9]:
report_data = []
for data, values in dash_data.items():
    entity_report = values.entity_non_strict_report
    entity_strict_report = values.entity_strict_report
    entity_report['Dataset'] = data
    entity_report['Scheme'] = 'IOB'
    entity_strict_report['Dataset'] = data
    entity_strict_report['Scheme'] = 'IOB2'
    report_data.append(pd.concat([
		entity_report, 
		entity_strict_report
	]))
df = pd.concat(report_data)


tag_mapping = {
    'PERS': 'PER'
}

dataset_mapping = {
    'ANERCorp_CamelLab_arabertv02': 'ANERCorp',
    'conll2003_bert': 'CoNLL-2003'
}

df['Tag'] = df['Tag'].replace(tag_mapping)
df['Dataset'] = df['Dataset'].replace(dataset_mapping)
# df = df[[col for col in df.columns if col != 'F1']].copy()
entity_report_data = df[~df['Tag'].isin(['micro', 'macro', 'weighted'])]

entity_report_data



# Assuming 'df_long' is your DataFrame reshaped
df_long = entity_report_data.melt(id_vars=["Tag", "Support", "Dataset", "Scheme"], 
                  value_vars=["Precision", "Recall"], 
                  var_name="Metric", value_name="Value")
df_long['Value'] = df_long['Value'].round(2)
# Creating the faceted bar plot using the reshaped data
fig = px.bar(df_long, x="Tag", y="Value",
             facet_row="Scheme", facet_col="Dataset",
             title="Precision and Recall Scores by Tag, Dataset, and Scheme",
             labels={"Value": "Score"},
             color="Metric", barmode="group",
             template="plotly_white",
             facet_row_spacing=0.15,  # Adjust to a higher value for more space
             facet_col_spacing=0.1,  # Adjust to a higher value for more space
             text='Value',  # Display the Value on top of each bar
            #  text_auto='.2s'  # Automatically format the text with 2 significant digits
             )
# fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')



# Update layout for better clarity
# fig.update_layout(
#     plot_bgcolor='rgba(255,255,255,1)',  # Ensure the background is white
#     paper_bgcolor='rgba(255,255,255,1)',  # Ensure the paper background is also white
# )

fig.show()


confusion heatmap

In [21]:
matrix_data = []
for data, values in dash_data.items():
    entity_matrix = pd.DataFrame(values.entity_non_strict_confusion_data['confusion_matrix']).T 
    entity_strict_matrix = pd.DataFrame(values.entity_strict_confusion_data['confusion_matrix']).T
    entity_matrix['Dataset'] = data
    entity_matrix['Scheme'] = 'IOB'
    entity_strict_matrix['Dataset'] = data
    entity_strict_matrix['Scheme'] = 'IOB2'
    matrix_data.append(pd.concat([
		entity_matrix, 
		entity_strict_matrix
	]))
    
matrix_df = pd.concat(matrix_data)
matrix_df.reset_index(inplace=True)
matrix_df.rename(columns={'index': 'Tag'}, inplace=True)
tag_mapping = {
    'PERS': 'PER'
}

dataset_mapping = {
    'ANERCorp_CamelLab_arabertv02': 'ANERCorp',
    'conll2003_bert': 'CoNLL-2003'
}

matrix_df['Tag'] = matrix_df['Tag'].replace(tag_mapping)
matrix_df['Dataset'] = matrix_df['Dataset'].replace(dataset_mapping)
# # df = df[[col for col in df.columns if col != 'F1']].copy()
# entity_report_data = df[~df['Tag'].isin(['micro', 'macro', 'weighted'])]
matrix_df



# Melt the DataFrame to align with Plotly requirements
df_final = matrix_df.melt(id_vars=['Tag', 'Dataset', 'Scheme'], value_vars=['TP', 'FP', 'FN'], 
                    var_name='Metric', value_name='Count')



# Assuming 'df_final' is your DataFrame prepared for plotting
unique_schemes = df_final['Scheme'].unique()
unique_datasets = df_final['Dataset'].unique()

# Create subplot configuration
fig = make_subplots(rows=len(unique_schemes), cols=len(unique_datasets),
                    subplot_titles=[f"{dataset} - {scheme}" for scheme in unique_schemes for dataset in unique_datasets],
                    shared_yaxes=True, horizontal_spacing=0.02, vertical_spacing=0.1)

# Determine the range of values to set a common scale
max_value = df_final['Count'].max()

# Add heatmaps
for idx, scheme in enumerate(unique_schemes):
    for jdx, dataset in enumerate(unique_datasets):
        filtered_data = df_final[(df_final['Scheme'] == scheme) & (df_final['Dataset'] == dataset)]
        heatmap_data = filtered_data.pivot_table(index='Metric', columns='Tag', values='Count', fill_value=0)
        text_data = filtered_data.pivot_table(index='Metric', columns='Tag', values='Count', fill_value=0).astype(int)

        
        
        fig.add_trace(
            go.Heatmap(
                z=heatmap_data,
                x=heatmap_data.columns,
                y=heatmap_data.index,
                colorscale='RdBu_r',
                coloraxis="coloraxis",  # Use a unified color axis
                text=text_data,  # Add text annotations
                texttemplate="%{text}",  # Use the text values directly
                hovertemplate="Metric: %{y}<br>Tag: %{x}<br>Count: %{text}<extra></extra>",
            ),
            row=idx + 1, col=jdx + 1
        )

# Update layout with a unified color scale and adjust the color bar
fig.update_layout(
    coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
    title_text="Confusion Matrix Metrics by Tag, Dataset, and Scheme",
    template="plotly_white",
    height=700, width=700,
)

fig.show()



bar confusion

In [33]:
matrix_data = []
for data, values in dash_data.items():
    entity_matrix = pd.DataFrame(values.entity_non_strict_confusion_data['confusion_matrix']).T 
    entity_strict_matrix = pd.DataFrame(values.entity_strict_confusion_data['confusion_matrix']).T
    entity_matrix['Dataset'] = data
    entity_matrix['Scheme'] = 'IOB'
    entity_strict_matrix['Dataset'] = data
    entity_strict_matrix['Scheme'] = 'IOB2'
    matrix_data.append(pd.concat([
		entity_matrix, 
		entity_strict_matrix
	]))
    
matrix_df = pd.concat(matrix_data)
matrix_df.reset_index(inplace=True)
matrix_df.rename(columns={'index': 'Tag'}, inplace=True)

matrix_df['Tag'] = matrix_df['Tag'].replace(tag_mapping)
matrix_df['Dataset'] = matrix_df['Dataset'].replace(dataset_mapping)
# Assuming 'matrix_df' has been set up as your DataFrame
df = matrix_df.copy()

# Calculate the sum of TP, FP, and FN per group for normalization purposes
grouped = df.groupby(['Tag', 'Dataset', 'Scheme']).sum()
grouped['Total'] = grouped['TP'] + grouped['FP'] + grouped['FN']

# Merge the total back into the original DataFrame
df = df.merge(grouped['Total'], on=['Tag', 'Dataset', 'Scheme'], how='left')

# Store actual counts for displaying as text
df['TP_Count'] = df['TP']
df['FP_Count'] = df['FP']
df['FN_Count'] = df['FN']

# Normalize the TP, FP, FN values for plotting
df['TP'] = df['TP'] / df['Total']
df['FP'] = df['FP'] / df['Total']
df['FN'] = df['FN'] / df['Total']

# Melt the DataFrame for plotting
df_long = df.melt(id_vars=["Tag", "Dataset", "Scheme"], value_vars=["TP", "FP", "FN"], var_name="Metric", value_name="Scale")
df_counts = df.melt(id_vars=["Tag", "Dataset", "Scheme"], value_vars=["TP_Count", "FP_Count", "FN_Count"], var_name="Metric", value_name="Count")

# Replace '_Count' to align with the other metric names
df_counts['Metric'] = df_counts['Metric'].str.replace('_Count', '')

# Merge percentage and actual count data
df_long = df_long.merge(df_counts, on=["Tag", "Dataset", "Scheme", "Metric"])

# Creating the faceted bar plot with actual counts displayed on normalized bars
fig = px.bar(df_long, x="Tag", y="Scale", color="Metric",
             facet_row="Scheme", facet_col="Dataset",
             title="Confusion Matrix Metrics by Tag, Dataset, and Scheme",
             labels={"Scale": "Scaled Counts"},
             barmode='group',
             template="plotly_white",
             facet_row_spacing=0.1,  # Adjusted spacing
             facet_col_spacing=0.08,
             text='Count'  # Display the actual Count on top of each bar
             )

# Set text to display above bars
# fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout(
    plot_bgcolor='rgba(255,255,255,1)',  # Ensure the background is white
    paper_bgcolor='rgba(255,255,255,1)',  # Ensure the paper background is also white
)

fig.show()
